In [ ]:
#@title Imports, initial setup (Ctrl+F9 to run all)
try:
    import gamry_parser
except:
    subprocess.run(
        ["pip", "install", "gamry-parser"], 
        encoding="utf-8", 
        shell=False)
finally:
    import gamry_parser

import os
import re
import pandas as pd
import matplotlib.pyplot as plt

z = gamry_parser.Impedance()

print('Done.')

In [ ]:
"""
### SCRIPT CONFIGURATION SETTINGS ###
"""
#@markdown **Load data** Parse Gamry *.DTA files from folder

#@markdown Where are the Gamry DTA files located?
file_path = "/path/to/gamry/files/" #@param {type:"string"}

#@markdown Which of the DTA files do we want to compare? (Regular expression matching)
file_pattern = "EIS" #@param {type:"string"}

#@markdown Which impedance frequencies should be shown? (separated by comma, e.g. `4, 1000, 10000`)
frequencies_to_show = "1, 5, 10, 10000" #@param {type:"string"}


frequencies_to_show = [int(val.strip()) for val in frequencies_to_show.split(",")]
files = [f for f in os.listdir(file_path) if 
    os.path.splitext(f)[1].lower() == ".dta" and
    len(re.findall(file_pattern, f)) > 0
    ]

# For repeating EIS, we need to properly sort files -- by chronological run-order instead of alphanumeric filename.
run_pattern = re.compile("[0-9]+_Run[0-9]+\.DTA", re.IGNORECASE)
files.sort(key=lambda fname: "_".join(["".join(filter(str.isdigit, x)).zfill(4) for x in run_pattern.search(fname).group().split("_")]))

if len(files) == 0:
    assert False, "No files matching the file filter [{}] were found.".format(file_pattern)
else:
    print('Found [{}] data files matching [{}]'.format(len(files), file_pattern))

# store aggregated start time, magnitude, phase, real, and imaginary impedance into separate variables
start_times = []
df_mag = pd.DataFrame()
df_phz = pd.DataFrame()
df_real = pd.DataFrame()
df_imag = pd.DataFrame()

# iterate through gamry files
index = 0
for dataf in files:
    name = os.path.splitext(dataf)[0].split('-')
    name = ", ".join(name[1:])
    
    # load file
    f = os.path.join(file_path, dataf)
    z.load(f)

    # process data header metadata
    header = z.get_header()
    start_time = pd.Timestamp("{} {}".format(header.get("DATE"), header.get("TIME")))
    print('{} [{}] ocp={}'.format(start_time, name, z.get_ocv_value()))
    
    # extract EIS curve
    res = z.get_curve_data()
    
    start_times.append(start_time)
    df_mag[name] = res['Zmod']
    df_phz[name] = res['Zphz']
    df_real[name] = res['Zreal']
    df_imag[name] = res['Zimag']

# post-processing for all collected curves

# validate the collected data, set frequency as dataframe index
df_mag["Freq"] = res["Freq"]
df_mag.set_index("Freq", inplace=True)
df_mag.mask(df_mag < 0, inplace=True)
df_phz["Freq"] = res["Freq"]
df_phz.set_index("Freq", inplace=True)
df_phz.mask(df_phz > 0, inplace=True)
df_phz.mask(df_phz < -90, inplace=True)
df_real["Freq"] = res["Freq"]
df_real.set_index("Freq", inplace=True)
df_real.mask(df_real < 0, inplace=True)
df_imag["Freq"] = res["Freq"]
df_imag.set_index("Freq", inplace=True)
df_imag.mask(df_imag > 0, inplace=True)
df_imag = df_imag.applymap(abs)


# print to screen impedance magnitude for the desired frequency
def freq_lookup(df, freq):
    return df.index.get_loc(freq, method='nearest')

for freq in frequencies_to_show:
    row_index = freq_lookup(df_mag, freq)
    print("\n    Showing Z_mag @ {} Hz [actual={:0.2f} Hz]".format(freq, df_mag.index[row_index]))
    print(df_mag.iloc[row_index])


In [ ]:
#@markdown **Bode Plot**: Display Zmag, Zphase vs. Freq
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

show_legend = False #@param{type:"boolean"}
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

data = []

# Yields a tuple of column name and series for each column in the dataframe
for (index, (columnName, columnData)) in enumerate(df_mag.iteritems()):
    newTrace = go.Scatter(
        x=df_mag.index,
        y=columnData,
        mode='lines',
        name=columnName,
        legendgroup=columnName,
        line=dict(color=DEFAULT_PLOTLY_COLORS[index % len(DEFAULT_PLOTLY_COLORS)]),
    )
    fig.add_trace(newTrace, row=1, col=1)

    newTrace = go.Scatter(
        x=df_mag.index,
        y=-1*df_phz[columnName],
        mode='lines',
        name=columnName,
        legendgroup=columnName,
        line=dict(color=DEFAULT_PLOTLY_COLORS[index % len(DEFAULT_PLOTLY_COLORS)]),
        showlegend=False
    )
    fig.add_trace(newTrace, row=2, col=1)

# variation = df_mag.std(axis=1) / newTrace['y']
# fig.add_trace({'x': df_mag.index, 'y': variation, 'name': 'Signal Variation'}, row=2, col=1)

layout = {
    'title': {'text': 'Bode Plot [{}]'.format(experiment_name),
              'yanchor': 'top',
              'y': 0.95,
              'x': 0.5 },
    'xaxis': {
        'anchor': 'x',
        'type': 'log'
    },
    'xaxis2': {
        'title': 'Frequency, Hz',
        'type': 'log',
        'matches': 'x'
    },
    'yaxis': {
        'title': 'Magnitude, Ohm',
        'type': 'log'
        ''
    },
    'yaxis2': {
        'title': 'Phase, deg',
    },
    'legend': {'x': 0.85, 'y': 0.97},
    'margin': dict(l=30, r=20, t=60, b=20),
    'width': 1200,
    'height': 500,
}
fig.update_layout(layout)
if not show_legend:
    fig.update_layout({"showlegend": False})

config={
    'displaylogo': False,
    'modeBarButtonsToRemove': ['select2d', 'lasso2d', 'hoverClosestCartesian', 'toggleSpikelines','hoverCompareCartesian']
}
fig.show(config=config)

In [ ]:
#@markdown **Polar Coordinate Plot**: Display -Zimag vs. Zreal

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

logx = True #@param {type:"boolean"}
logy = True #@param {type:"boolean"}

fig = make_subplots(rows=1, cols=1,vertical_spacing=0.02, horizontal_spacing=0.02)

data = []
# Yields a tuple of column name and series for each column in the dataframe
for (index, columnName) in enumerate(df_real.columns):
    newTrace = go.Scatter(
        x=df_real[columnName],
        y=-df_imag[columnName],
        mode='markers+lines',
        name=columnName,
        legendgroup=columnName,
        text='Freq: ' + df_real.index.astype(str),
        line=dict(color=DEFAULT_PLOTLY_COLORS[index % len(DEFAULT_PLOTLY_COLORS)]),
    )
    fig.add_trace(newTrace, row=1, col=1)

# variation = df_mag.std(axis=1) / newTrace['y']
# fig.add_trace({'x': df_mag.index, 'y': variation, 'name': 'Signal Variation'}, row=2, col=1)

layout = {
    'title': {'text': 'Impendance Plot, Polar Coord. [tests matching: {}]'.format(file_pattern),
              'yanchor': 'top',
              'y': 0.95,
              'x': 0.5 },
    'xaxis': {
        'anchor': 'x',
        'title': 'Zreal, Ohm',
        'type': 'log' if logx else 'linear',
    },
    'yaxis': {
        'title': '-Zimag, Ohm',
        'type': 'log' if logy else 'linear',
    },
    'legend': {'x': 0.03, 'y': 0.97},
    'margin': dict(l=30, r=20, t=60, b=20),
    'width': 600,
    'height': 600,
}
fig.update_layout(layout)
config={
    'displaylogo': False,
    'modeBarButtonsToRemove': ['select2d', 'lasso2d', 'hoverClosestCartesian', 'toggleSpikelines','hoverCompareCartesian']
}
fig.show(config=config)

In [ ]:
#@markdown **Time-series Plot (Z_mag)**
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

impedance_type = "magnitude" #@param ["magnitude", "phase", "real", "imaginary"]

#@markdown Which impedance frequencies should be shown? (separated by comma, e.g. `4, 1000, 10000`)
frequencies_to_show = "3,5,5000" #@param {type:"string"}

impedance_map = dict(magnitude=df_mag, phase=df_phz, real=df_real, imaginary=df_imag)
impedance_yaxis_map = {
    "magnitude": {'title': 'Impedance Magnitude, Ohm', 'type': 'log'},
    "phase": {'title': 'Phase Angle, deg', 'type': 'linear'},
    "real": {'title': 'Real Impedance, Ohm', 'type': 'log'},
    "imaginary": {'title': '- Imaginary Impedance, Ohm', 'type': 'log'},
}
frequencies_to_show = [int(val.strip()) for val in frequencies_to_show.split(",")]


def freq_lookup(df, freq):
    return df.index.get_loc(freq, method='nearest')

ilocs = [freq_lookup(df_mag, freq) for freq in frequencies_to_show]

source = impedance_map.get(impedance_type)
source_yaxis_config = impedance_yaxis_map.get(impedance_type)

# df_mag.T.to_csv("magnitude_transpose.csv")
df_time = source.copy()
df_time.columns = start_times
df_time.T.to_csv("longitudinal.csv")

df_time = source.copy().iloc[ilocs]
df_time.columns = start_times
df_time = df_time.T
df_time.to_csv("longitudinal-filtered.csv")

fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02)
# Yields a tuple of column name and series for each column in the dataframe
for (index, (columnName, columnData)) in enumerate(df_time.iteritems()):
    newTrace = go.Scatter(
        x=df_time.index,
        y=columnData,
        mode='lines',
        name="{} Hz".format(round(columnName,1)),
        legendgroup=columnName,
        line=dict(color=DEFAULT_PLOTLY_COLORS[index % len(DEFAULT_PLOTLY_COLORS)]),
    )
    fig.add_trace(newTrace, row=1, col=1)

layout = {
    'title': {'text': 'Time-series Plot [{}]'.format(experiment_name),
              'yanchor': 'top',
              'y': 0.95,
              'x': 0.5 },
    'xaxis': {
        'anchor': 'x',
        # 'type': 'log'
    },
    # 'xaxis2': {
    #     'title': 'Frequency, Hz',
    #     # 'type': 'log',
    #     'matches': 'x'
    # },
    'yaxis': source_yaxis_config,
    # 'yaxis2': {
    #     'title': 'Phase, deg',
    # },
    'legend': {'x': 0.85, 'y': 0.97},
    'margin': dict(l=30, r=20, t=60, b=20),
    'width': 1200,
    'height': 500,
}
fig.update_layout(layout)

config={
    'displaylogo': False,
    'modeBarButtonsToRemove': ['select2d', 'lasso2d', 'hoverClosestCartesian', 'toggleSpikelines','hoverCompareCartesian']
}
fig.show(config=config)

In [ ]:
#@markdown WORK IN PROGRESS **Simulation**: Fit collected data to an electrochemical model
equivalent_model = "Body Impedance Model" #@param ["Randles", "Randles + Diffusion", "Randles + Corrosion", "Body Impedance Model"]

# import additional math libs
import numpy as np
from numpy import pi, sqrt
try:
    from lmfit import Model
except:
    subprocess.run(
        ["pip", "install", "--upgrade", "lmfit"], 
        encoding="utf-8", 
        shell=False)
finally:
    from lmfit import Model

  
def reference_model(freq, Rct, Cdl_C, Cdl_a, Rsol):
    # modifeid randle circuit -- The dual layer capacitance is non-ideal due to 
    # diffusion-related limitations. It has been replaced with a constant phase
    # element. Circuit layout: SERIES(Rsol, PARALLEL(Rct, CPE))
    CPE1 = 1/(Cdl_C*(1j*2*pi*freq)**(Cdl_a))
    
    return 1/((1/Rct) + (1/CPE1)) + Rsol

def body_impedance_model(freq, R1, C1, R2, C2, R3, C3, Rs, Cs): # P1, R2, C2, P2, R3, C3, P3, Rs, Cs):
    # Layer1 = 1/((1/R1) + (C1*(1j*2*pi*(freq + P1))))
    # Layer2 = 1/((1/R2) + (C2*(1j*2*pi*(freq + P2))))
    # Layer3 = 1/((1/R3) + (C3*(1j*2*pi*(freq + P3))))
    Layer1 = 1/((1/R1) + (C1*(1j*2*pi*(freq))))
    Layer2 = 1/((1/R2) + (C2*(1j*2*pi*(freq))))
    Layer3 = 1/((1/R3) + (C3*(1j*2*pi*(freq))))
    Zc_s = 1/(Cs * 1j * 2 * pi * (freq))
    Zr_s = Rs
    return Zc_s + Zr_s + Layer1 + Layer2 + Layer3

def diffusion_model(freq, Rct, Cdl_C, Cdl_a, Rsol, Zdf):
    # A modified Randle circuit with a warburg component included.
    # Circuit layout: SERIES(Rsol, PARALLEL(SERIES(Warburg, Rct), CPE))
    CPE1 = 1/(Cdl_C*(1j*2*pi*freq)**(Cdl_a))
    
    ## use finite length diffusion constant 
    # Warburg = Zdf * (np.tanh(sqrt(2j*pi*freq*TCdf))/sqrt(2j*pi*freq*TCdf))
    
    ## use infinite warburg coeff (simplified)
    Warburg = 1/(Zdf*sqrt(1j*2*pi*freq))
    
    return 1/((1/(Rct + Warburg)) + (1/CPE1)) + Rsol

def corrosion_model(freq, Rc, Cdl_C, Cdl_a, Rsol, Ra, La):
    # split cathodic and anodic resistances with inductive component
    CPE1 = 1/(Cdl_C*(1j*2*pi*freq)**(Cdl_a))
    
    Za   = Ra + 2j*pi*freq*La
    Rct = 1 / ((1/Rc) + (1/Za))
    
    return 1/((1/Rct) + (1/CPE1)) + Rsol

def corrosion2_model(freq, Rc, Cdl_C, Cdl_a, Rsol, Ra, La):
    # split cathodic and anodic resistances with inductive component
    CPE1 = 1/(Cdl_C*(1j*2*pi*freq)**(Cdl_a))
    
    Za   = Ra + 2j*pi*freq*La
    Rct = 1 / ((1/Rc) + (1/Za))
    
    return 1/((1/Rct) + (1/CPE1)) + Rsol

# create the model
if equivalent_model == "Body Impedance Model":
    # model a membrane as resistor and capacitor in parallel.
    gmodel = Model(body_impedance_model)
    gmodel.set_param_hint('C1', value = 85e-9, min = 1e-9, max=1e-6)
    gmodel.set_param_hint('C2', value = 85e-9, min = 1e-9, max=1e-6)
    gmodel.set_param_hint('C3', value = 85e-9, min = 1e-9, max=1e-6)
    gmodel.set_param_hint('R1', value = 45e3, min=1e3, max=1e6)
    gmodel.set_param_hint('R2', value = 875e3, min=1e3, max=1e6)
    gmodel.set_param_hint('R3', value = 750, min=0, max=1e3)
    gmodel.set_param_hint('Rs', value = 400, min=200, max=600)
    gmodel.set_param_hint('Cs', value = 150e-9, min=50e-9, max=5e-6)
    
    
elif equivalent_model == 'Randles':
    # default, use a randle's circuit with non-ideal capacitor assumption
    gmodel = Model(reference_model)
    gmodel.set_param_hint('Rct', value = 1e7, min = 1e3, max = 1e9)

elif equivalent_model == 'Randles + Diffusion':
    # use previous model and add a warburg
    gmodel = Model(diffusion_model)
    gmodel.set_param_hint('Rct', value = 1e6, min = 1e3, max = 1e10)
    gmodel.set_param_hint('Zdf', value = 1e4, min = 1e3, max = 1e6)

else: 
    # Randle + Corrosion
    gmodel = Model(corrosion_model)
    gmodel.set_param_hint('Rc', value = 5e5, min = 1e3, max = 1e10)
    gmodel.set_param_hint('Ra', value = 5e5, min = 1e3, max = 1e10)
    gmodel.set_param_hint('La', value = 1e6, min = 0, max = 1e9)
  
# initial guess shared across all models, with defined acceptable limits
gmodel.set_param_hint('Cdl_C', value = 5e-5, min = 1e-12)
gmodel.set_param_hint('Cdl_a', value = 0.9, min = 0, max = 1)
gmodel.set_param_hint('Rsol', value = 1000, min = 100, max = 5e5)

# now solve for each loaded sensor
a = []
freq = np.asarray(df_mag.index)
for (index, columnName) in enumerate(df_mag.columns):
    print('Model Simulation [{}] on [{}]'.format(equivalent_model, columnName))
    impedance = np.asarray(df_real[columnName]) - 1j * np.asarray(df_imag[columnName])
    
    # fit_weights = (np.arange(len(freq))**.6)/len(freq) #weight ~ freq
    fit_weights = np.ones(len(freq))/len(freq) #equal weight
    
    result = gmodel.fit(impedance, freq=freq, weights = fit_weights)
    print(result.fit_report(show_correl=False))

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
    data = []
    # Yields a tuple of column name and series for each column in the dataframe
    rawTrace = go.Scatter(
        x=freq,
        y=df_mag[columnName],
        mode='markers',
        name=columnName,
        legendgroup='raw',
        marker=dict(color=DEFAULT_PLOTLY_COLORS[0]),
    )
    fig.add_trace(rawTrace, row=1, col=1)
    fitTrace = go.Scatter(
        x=freq,
        y=np.abs(result.best_fit),
        mode='lines',
        name=columnName,
        legendgroup='model',
        line=dict(color=DEFAULT_PLOTLY_COLORS[1]),
    )
    fig.add_trace(fitTrace, row=1, col=1)
    rawTrace = go.Scatter(
        x=freq,
        y=-df_phz[columnName],
        mode='markers',
        name=columnName,
        legendgroup='raw',
        showlegend=False,
        marker=dict(color=DEFAULT_PLOTLY_COLORS[0]),
    )
    fig.add_trace(rawTrace, row=2, col=1)
    fitTrace = go.Scatter(
        x=freq,
        y=-180/pi*np.angle(result.best_fit),
        mode='lines',
        name=columnName,
        legendgroup='model',
        showlegend=False,
        line=dict(color=DEFAULT_PLOTLY_COLORS[1]),
    )
    fig.add_trace(fitTrace, row=2, col=1)
    layout = {
        'title': 'Model Fit [{}] for [{}]'.format(equivalent_model, columnName),
        'xaxis': {
            'anchor': 'x',
            'type': 'log'
        },
        'xaxis2': {
            'anchor': 'x',
            'type': 'log'
        },
        'yaxis': {
            'type': 'log'
        },
        'legend': {'x': 0.85, 'y': 0.97},
        'margin': dict(l=30, r=20, t=60, b=20),
        'width': 1200,
        'height': 500,
    }
    
    fig.update_layout(layout)
    fig.show()